In [ ]:
#coding=utf-8
import pandas as pd
from math import *
import pickle


In [ ]:
shop = pd.read_csv('../ants_data/训练数据-ccf_first_round_shop_info.csv')
train = pd.read_csv('../ants_data/训练数据-ccf_first_round_user_shop_behavior.csv')
test = pd.read_csv('../ants_data/AB榜测试集-evaluation_public.csv')

In [ ]:
print (list(shop.ix[:,'mall_id'].unique()))

In [ ]:
def haversine(lon1, lat1, lon2, lat2): # 经度1，纬度1，经度2，纬度2 （十进制度数）  
    """ 
    Calculate the great circle distance between two points  
    on the earth (specified in decimal degrees) 
    """  
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])  
    dlon = lon2 - lon1   
    dlat = lat2 - lat1   
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2  
    c = 2 * asin(sqrt(a))   
    r = 6371 
    return c * r * 1000

In [ ]:
def calcDistance(Lat_A, Lng_A, Lat_B, Lng_B):
    """
        # input Lat_A 纬度A
        # input Lng_A 经度A
        # input Lat_B 纬度B
        # input Lng_B 经度B
        # output distance 距离(km)
    """
    ra = 6378.140  # 赤道半径 (km)
    rb = 6356.755  # 极半径 (km)
    flatten = (ra - rb) / ra  # 地球扁率
    rad_lat_A, rad_lng_A, rad_lat_B, rad_lng_B = map(radians, [Lat_A, Lng_A, Lat_B, Lng_B])  
    pA = atan(rb / ra * tan(rad_lat_A))
    pB = atan(rb / ra * tan(rad_lat_B))
    xx = acos(sin(pA) * sin(pB) + cos(pA) * cos(pB) * cos(rad_lng_A - rad_lng_B))
    print('pA',Lng_A,Lat_A,'pB',Lat_B,Lng_B)
    c1 = (sin(xx) - xx) * (sin(pA) + sin(pB)) ** 2 / cos(xx / 2) ** 2
    c2 = (sin(xx) + xx) * (sin(pA) - sin(pB)) ** 2 / sin(xx / 2) ** 2
    dr = flatten / 8 * (c1 - c2)
    distance = ra * (xx + dr)
    return distance

In [ ]:
Lat_A=32.060255; Lng_A=118.796877
Lat_B=39.904211; Lng_B=116.407395
distance=calcDistance(Lat_A,Lng_A,Lat_B,Lng_B)
print('(Lat_A, Lng_A)=({0:10.3f},{1:10.3f})'.format(Lat_A,Lng_A))
print('(Lat_B, Lng_B)=({0:10.3f},{1:10.3f})'.format(Lat_B,Lng_B))
print('Distance={0:10.3f} km'.format(distance))

In [ ]:
def get_nearest_shop_id(longitude, latitude, shop_ids_by_mall_id):
    #map_data = map(lambda lat, lng: calcDistance(lat, lng, latitude, longitude), shop_ids_by_mall_id['latitude'], shop_ids_by_mall_id['longitude'])
    map_data = map(lambda lng, lat: haversine(lng, lat, longitude, latitude), shop_ids_by_mall_id['longitude'], shop_ids_by_mall_id['latitude'])
    data = list(map_data)
    shop_ids_by_mall_id = shop_ids_by_mall_id.reset_index(drop=True)
    return min(data),shop_ids_by_mall_id.ix[data.index(min(data))]['shop_id']

In [ ]:
test['shop_id']='s_xxx'
n_samples = test.shape[0]
for i in range(n_samples):
    mall_id = test.ix[i]['mall_id']
    shop_ids_by_mall_id = shop[shop.mall_id == mall_id]
    min_dist, nn_shop_id = get_nearest_shop_id(test.ix[i]['longitude'], test.ix[i]['latitude'], shop_ids_by_mall_id)
    test.loc[i,'shop_id']= nn_shop_id
    print ("sample_num: ", i, "min_dist:", min_dist, " shop_id:", test.ix[i]['shop_id'])
test.ix[:,['row_id','shop_id']].to_csv("results.csv",index=False)

In [ ]:
test['shop_id']='s_xxx'
mall_ids = list( shop.ix[:,'mall_id'].unique() )
for mall_id in mall_ids:
        mall_id = 'm_5085'
        shop_ids_by_mall_id = shop[shop.mall_id == mall_id]
        
        #print(shop_ids_by_mall_id)
        data_with_mall_id = pd.merge(shop_ids_by_mall_id, train.ix[:,['wifi_infos','shop_id']],on='shop_id')
        
        # get all shops
        shops = list(data_with_mall_id.ix[:,'shop_id'].unique())
        
        # get all wifis
        wifis = set()
        for wifi in data_with_mall_id.ix[:,'wifi_infos']:
            for _wifi in wifi.split(';'):
                wifis.add(_wifi.split('|')[0])
        wifis = list(wifis)
        print(len(wifis))
        with open('data/'+str(mall_id)+'_shops.pkl','wb') as f:
                pickle.dump(shops,f)
        with open('data/'+str(mall_id)+'_wifis.pkl','wb') as f:
                pickle.dump(wifis,f)
                
        with open('data/'+str(mall_id)+'_shops.pkl','rb') as f:
                shops = pickle.load(f)
                print(shops)
        print(len(shops))
        data_with_mall_id.drop(['category_id','longitude','latitude','price','mall_id'], axis=1, inplace=True)
        
        # add more features into wifis here
        new_X = pd.DataFrame(columns=range(10))
        new_Y = list()
        _n_samples = data_with_mall_id.shape[0]
        for i in range(_n_samples):
            print(i,_n_samples)
            shop_id = data_with_mall_id.ix[i]['shop_id']
            new_Y.append(shops.index(shop_id))
            wifi_infos = data_with_mall_id.ix[i]['wifi_infos'].split(';')
            for j in range(len(wifi_infos)):
                _wifi = wifi_infos[j].split('|')
                new_X.at[i,j]=str(wifis.index(_wifi[0]))+':'+str(_wifi[1])
        new_Y = pd.DataFrame(new_Y,columns=['shop_id'])
        train_data_by_mall_id = pd.concat([new_Y, new_X], axis=1)
        # shuffle samples
        train_data_by_mall_id=train_data_by_mall_id.sample(frac=1)
        train_data_by_mall_id.to_csv("data/"+str(mall_id)+'.csv',index=False,header=None)
        break

In [ ]:
import xgboost as xgb
import numpy as np
from sklearn.datasets import load_svmlight_file
raw_data = load_svmlight_file('m_5085.csv')

train = raw_data[0].toarray()
label = raw_data[1]

sz = train.shape

kfolds = 0.7

train_X = train[:int(sz[0] * kfolds), :]
test_X = train[int(sz[0] * kfolds):, :]

train_Y = label[:int(sz[0] * kfolds)]
test_Y = label[int(sz[0] * kfolds):]

print(train_Y.shape)
print(test_Y.shape)
xg_train = xgb.DMatrix(train_X, label=train_Y)
xg_test = xgb.DMatrix(test_X, label=test_Y)

#num_class = dtrain.get_label().shape[0]
#params = {'max_depth':3,'num_class':num_class,'objective':'multi:softmax','silent':1}
#num_boost_round = 3
#model = xgb.train(params, dtrain, num_boost_round)

In [82]:
import pandas as pd
import numpy as np
train = pd.read_csv('data/m_4543.csv', header=None)
sz = train.shape
print(sz)
for i in range(sz[0]):
    nan_number = train.ix[i,1:][train.ix[i,1:].isnull()].shape[0]
    ordered_list=sorted(train.ix[i,1:][~train.ix[i,1:].isnull()],key=lambda data: int(data.split(':')[0]))
    # filter duplicate 
    filtered_ordered_list = []
    counter = Counter([elem.split(':')[0] for elem in ordered_list])
    for k,v in counter.items():
        _list = [elem for elem in ordered_list if elem.split(':')[0] == k]
        if v == 1:
            filtered_ordered_list.extend(_list)
        if v > 1:
            nan_number += v - 1
            filtered_ordered_list.append(_list[0])
    for j in range(nan_number):
        filtered_ordered_list.append('NaN')
    train.ix[i,1:]= filtered_ordered_list
print(train)

(2, 11)
   0        1         2         3         4         5         6         7   \
0   3  732:-75  1132:-64  1314:-54  1582:-78  2393:-80  3647:-27  3717:-80   
1  57  544:-72   732:-38  1132:-86  1461:-91  1822:-89  2140:-38  2224:-74   

         8         9         10  
0  3934:-80       NaN       NaN  
1  3245:-35  3647:-89  3717:-74  


In [81]:
from collections import Counter
tokens = ['2393:-75', '1132:-64', '1314:-54', '1314:-78', '2393:-80']
print(tokens)
data = []
nan_number = 0
counter = Counter([elem.split(':')[0] for elem in tokens])
for k,v in counter.items():
    _list = [token for token in tokens if token.split(':')[0] == k]
    print(_list)
    if v == 1:
        data.extend(_list)
    if v > 1:
        nan_number += v - 1
        data.append(_list[0])
for i in range(nan_number):
    data.append('NaN')
print(data,nan_number)

['2393:-75', '1132:-64', '1314:-54', '1314:-78', '2393:-80']
['2393:-75', '2393:-80']
['1132:-64']
['1314:-54', '1314:-78']
['2393:-75', '1132:-64', '1314:-54', 'NaN', 'NaN'] 2
